# Tree Audit
### outputs
* Tree Audit

In [1]:
import pandas as pd
import os
import glob
import numpy as np

In [2]:
path1 = r"C:\Users\jacque.trahan\Downloads\Central Division 2021 Debris Removal Tracker"
file_type1 = ".xlsx"
loadit1 = glob.glob(path1 + file_type1)
df1 = pd.read_excel(max(loadit1, key=os.path.getctime))

In [3]:
df1.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 138 columns):
 #    Column                                              Dtype         
---   ------                                              -----         
 0    APN_ROW Segment                                     object        
 1    ROE Type                                            object        
 2    Structural Status                                   object        
 3    Haz Tree Status                                     object        
 4    Debris Crew                                         object        
 5    Tree Crew                                           float64       
 6    Street #                                            object        
 7    Street Name                                         object        
 8    Unit #                                              object        
 9    City                                                object        
 10   County      

In [4]:
smartsheets = df1.copy()
smartsheets = smartsheets[['APN_ROW Segment', 'Street #', 'Street Name', 'Structural Status','County',
                           'Haz Trees Assessment', 'Number of Trees']]

# Agustin Emails

In [5]:
path2 = r"C:\Users\jacque.trahan\Downloads\2021-12-17_Central Property Summary_CalRecycle Dixie Fire*"
file_type2 = ".xlsx"
loadit2 = glob.glob(path2 + file_type2)
df2 = pd.read_excel(max(loadit2, key=os.path.getctime), sheet_name=1)

In [6]:
# df2.info(verbose=True)

In [7]:
Agustin = df2.copy()
Agustin = Agustin[['APN', 'Tree Survey Date', 'Number of Trees']]

# Merge

In [8]:
final = smartsheets.merge(Agustin,
                         left_on="APN_ROW Segment",
                         right_on="APN",
                         how="outer",
                         suffixes=("_SmartSheets", "_Agustin"))

In [9]:
def compare(final, comp1, comp2):
    if final[comp1] == final[comp2]:
        return 1
    else:
        return 0

In [10]:
final = final[['APN_ROW Segment', 'APN', 'Street #', 'Street Name','Structural Status', 'County',
          'Haz Trees Assessment', 'Tree Survey Date',
          'Number of Trees_SmartSheets', 'Number of Trees_Agustin']]

In [11]:
final = final.copy()
final["Haz Trees Assessment"] = pd.to_datetime(final["Haz Trees Assessment"]).dt.date.fillna(0)
final["Tree Survey Date"] = pd.to_datetime(final["Tree Survey Date"]).dt.date.fillna(0)

In [12]:
final.fillna({'Number of Trees_SmartSheets':0,
             'Number of Trees_Agustin':0}, inplace=True)

In [13]:
final.insert(final.columns.get_loc('Tree Survey Date')+1, 'Date_Match',
            final.apply(compare, comp1='Haz Trees Assessment', comp2='Tree Survey Date', axis=1))

In [14]:
final.insert(final.columns.get_loc('Number of Trees_Agustin')+1, 'Number match',
            final.apply(compare, comp1='Number of Trees_SmartSheets', comp2='Number of Trees_Agustin', axis=1))

In [15]:
final_style = final.style.set_properties(**{'background-color': 'yellow'},
                                        subset=['Date_Match', 'Number match'])

In [16]:
final_style.to_excel("Agustin email checks.xlsx")